<a href="https://colab.research.google.com/github/sakurag118/ELECTIVA-TECNICA-I-C-DATOS-/blob/main/SQLite_y_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller: Gestión de Inventario con SQLite y Python
**Estudiante:** Yuliet Sakura Garcia Fonseca  
**Materia:** Ciencia de Datos I  
**Fecha:** (poner la fecha)  

**Objetivo:** Implementar una base de datos SQLite para gestionar inventario usando Python. Crear clases básicas, insertar, consultar, actualizar y eliminar productos. Exportar resultados a DataFrame con pandas.

**Instrucciones:** Ejecutar las celdas de código en orden. Leer las explicaciones en cada celda para entender qué hace cada línea.

In [1]:
# Librerías necesarias
import sqlite3        # para la base de datos SQLite
import pandas as pd   # para manejar tablas (dataframes)
from datetime import datetime
import random         # para crear datos de prueba


Clase Product (modelo sencillo)

In [2]:
# Clase que representa un producto en Python
class Product:
    def __init__(self, nombre, categoria, precio, cantidad, proveedor, fecha_ingreso=None):
        # __init__ se ejecuta cuando creas un producto: Product(...)
        # 'self' es la referencia al objeto que estamos creando
        self.nombre = nombre
        self.categoria = categoria
        self.precio = float(precio)      # guardamos como número decimal
        self.cantidad = int(cantidad)    # guardamos como entero
        self.proveedor = proveedor
        # si no das fecha, ponemos la fecha actual en formato ISO
        self.fecha_ingreso = fecha_ingreso or datetime.now().strftime("%Y-%m-%d")

    def __repr__(self):
        # Esto define cómo se ve el producto si lo imprimes
        return f"Product(nombre={self.nombre}, categoria={self.categoria}, precio={self.precio}, cantidad={self.cantidad}, proveedor={self.proveedor}, fecha={self.fecha_ingreso})"


Clase InventoryDB (manejo de SQLite)

In [3]:
class InventoryDB:
    def __init__(self, db_name="productos.db"):
        # Abrimos (o creamos si no existe) la base de datos
        self.db_name = db_name
        self.conn = sqlite3.connect(self.db_name)  # Conexión a archivo .db
        self.conn.row_factory = sqlite3.Row       # Para obtener resultados como diccionarios
        self.c = self.conn.cursor()               # Cursor: nos permite ejecutar comandos SQL
        self.create_table()                       # Crear tabla si no existe

    def create_table(self):
        # CREAMOS LA TABLA si no existe
        self.c.execute("""
            CREATE TABLE IF NOT EXISTS inventario (
                id INTEGER PRIMARY KEY,
                nombre TEXT,
                categoria TEXT,
                precio REAL,
                cantidad INTEGER,
                proveedor TEXT,
                fecha_ingreso TEXT
            )
        """)
        self.conn.commit()  # Guardar cambios

    def insert_product(self, product):
        # Inserta un producto (objeto Product)
        sql = "INSERT INTO inventario (nombre, categoria, precio, cantidad, proveedor, fecha_ingreso) VALUES (?, ?, ?, ?, ?, ?)"
        params = (product.nombre, product.categoria, product.precio, product.cantidad, product.proveedor, product.fecha_ingreso)
        self.c.execute(sql, params)   # Usamos marcadores '?' para seguridad (previene SQL injection)
        self.conn.commit()
        return self.c.lastrowid      # devuelve el id del registro insertado

    def insert_many(self, list_of_products):
        # Inserta muchos productos a la vez usando executemany (más rápido)
        sql = "INSERT INTO inventario (nombre, categoria, precio, cantidad, proveedor, fecha_ingreso) VALUES (?, ?, ?, ?, ?, ?)"
        params = [(p.nombre, p.categoria, p.precio, p.cantidad, p.proveedor, p.fecha_ingreso) for p in list_of_products]
        self.c.executemany(sql, params)
        self.conn.commit()

    def get_all(self):
        # Devuelve todos los productos como lista de dicts
        self.c.execute("SELECT * FROM inventario")
        rows = self.c.fetchall()
        return [dict(row) for row in rows]

    def query(self, where_clause="", params=()):
        # Consulta personalizada (ej: "WHERE precio > ?", (100,))
        sql = "SELECT * FROM inventario " + where_clause
        self.c.execute(sql, params)
        rows = self.c.fetchall()
        return [dict(row) for row in rows]

    def update_product(self, product_id, **kwargs):
        # Actualiza campos pasados por kwargs, por ejemplo: update_product(3, precio=50, cantidad=10)
        fields = []
        values = []
        for k, v in kwargs.items():
            fields.append(f"{k} = ?")
            values.append(v)
        values.append(product_id)  # para WHERE
        sql = f"UPDATE inventario SET {', '.join(fields)} WHERE id = ?"
        self.c.execute(sql, tuple(values))
        self.conn.commit()
        return self.c.rowcount   # cuántas filas fueron afectadas

    def delete_product(self, product_id):
        self.c.execute("DELETE FROM inventario WHERE id = ?", (product_id,))
        self.conn.commit()
        return self.c.rowcount

    def to_dataframe(self, where_clause="", params=()):
        # Devuelve un pandas DataFrame de la consulta
        query = "SELECT * FROM inventario " + where_clause
        return pd.read_sql_query(query, self.conn, params=params)

    def close(self):
        self.conn.close()


Crear datos de prueba (rápido)

In [4]:
# Función para crear N productos de prueba
def crear_datos_prueba(n=10):
    categorias = ["Electronica", "Ropa", "Alimentos", "Hogar"]
    proveedores = ["Proveedor1", "Proveedor2", "Proveedor3"]
    lista = []
    for i in range(n):
        nombre = f"Producto_{i+1}"
        categoria = random.choice(categorias)
        precio = round(random.uniform(10, 1000), 2)
        cantidad = random.randint(1, 100)
        proveedor = random.choice(proveedores)
        p = Product(nombre, categoria, precio, cantidad, proveedor)
        lista.append(p)
    return lista


Ejemplo: usar todo paso a paso

In [5]:
# 1) Crear la base de datos (archivo productos.db en el entorno de Colab)
db = InventoryDB("productos.db")  # se crea o abre

# 2) Insertar un solo producto
p1 = Product("Camisa Azul", "Ropa", 25.50, 10, "Proveedor1")
id_insertado = db.insert_product(p1)
print("Insertado id:", id_insertado)

# 3) Insertar varios productos
lista = crear_datos_prueba(20)
db.insert_many(lista)
print("Insertados 20 productos de prueba.")

# 4) Consultar todos y mostrar
todos = db.get_all()
print("Cantidad total de registros:", len(todos))
print("Primeros 3 registros:", todos[:3])

# 5) Consulta con condición: productos del proveedor1
prov1 = db.query("WHERE proveedor = ?", ("Proveedor1",))
print("Productos de Proveedor1:", len(prov1))

# 6) Actualizar un producto (ejemplo: cambiar precio y cantidad)
filas = db.update_product(id_insertado, precio=30.0, cantidad=12)
print("Filas actualizadas:", filas)

# 7) Borrar un producto (ejemplo)
# filas_borradas = db.delete_product(id_insertado)
# print("Filas borradas:", filas_borradas)

# 8) Cerrar
# db.close()


Insertado id: 1
Insertados 20 productos de prueba.
Cantidad total de registros: 21
Primeros 3 registros: [{'id': 1, 'nombre': 'Camisa Azul', 'categoria': 'Ropa', 'precio': 25.5, 'cantidad': 10, 'proveedor': 'Proveedor1', 'fecha_ingreso': '2025-10-23'}, {'id': 2, 'nombre': 'Producto_1', 'categoria': 'Hogar', 'precio': 175.45, 'cantidad': 35, 'proveedor': 'Proveedor2', 'fecha_ingreso': '2025-10-23'}, {'id': 3, 'nombre': 'Producto_2', 'categoria': 'Ropa', 'precio': 247.98, 'cantidad': 46, 'proveedor': 'Proveedor1', 'fecha_ingreso': '2025-10-23'}]
Productos de Proveedor1: 7
Filas actualizadas: 1
